In [68]:
# Volatility calculation for a list of Index

In [ ]:
#!pip install ipywidgets

In [70]:
import pandas as pd
from scipy.stats import skew, kurtosis
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

tickers = ["^SPX", "^IXIC", "^DJI", "^VIX","SPY","^RUT","CL=F","GC=F","CME"] # Stock is CME
results = []

for ticker in tickers:
    data = yf.download(ticker, start="2023-05-19", end="2023-08-19", interval="1d")
    returns = data['Close'].pct_change().dropna()
    drawdown = returns - returns.cummax()

    # Compute the risk measures
    measures = {
        'Ticker': ticker,
        'Standard Deviation': returns.std(),
        'Mean Absolute Deviation': returns.mad(),
        'Semi Standard Deviation': returns[returns < 0].std(),
        'First Lower Partial Moment': returns[returns < 0].mean(),
        'Second Lower Partial Moment': returns[returns < 0].apply(lambda x: x**2).mean(),
        'Value at Risk': returns.quantile(0.05),
        'Conditional Value at Risk': returns[returns <= returns.quantile(0.05)].mean(),
        'Skewness': skew(returns),
        'Kurtosis': kurtosis(returns),
        'Ulcer Index': (drawdown**2).mean()**0.5,
        'Average Drawdown': drawdown.mean(),
        'Max Drawdown': drawdown.min(),
        'Tail Gini of Losses': sum(sorted(returns)[:int(len(returns) * 0.05)]) / sum(sorted(returns)),
    }

    # Entropic Value at Risk
    alpha = 0.05
    sorted_returns = sorted(returns)
    threshold_index = int(alpha * len(sorted_returns))
    threshold = sorted_returns[threshold_index]
    tail_mean = sum(sorted_returns[:threshold_index]) / threshold_index
    measures['Entropic Value at Risk'] = threshold + (1 / (1 - alpha)) * tail_mean

    # Add the results to a list
    results.append(measures)

# Create a DataFrame from the results
measures_df = pd.DataFrame(results)

# Set the index to 'Ticker' column and transpose the DataFrame
measures_df.set_index('Ticker', inplace=True)
transposed_measures_df = measures_df.transpose()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [71]:
from IPython.display import display
import ipywidgets as widgets
# Create a function to display the DataFrame based on the sorted column
def display_table(sort_by):
    sorted_df = transposed_measures_df.sort_values(by=sort_by)
    display(sorted_df)
# Create a dropdown widget with columns to sort by
dropdown = widgets.Dropdown(
    options=transposed_measures_df.columns,
    description='Sort by:',
    disabled=False,
)
print("                                      RISK MEASURES OF THE INDICES WITH RESPECT TO THE TICKER")
# Create a widget to interact with the dropdown
widgets.interactive(display_table, sort_by=dropdown)

                                      RISK MEASURES OF THE INDICES WITH RESPECT TO THE TICKER


interactive(children=(Dropdown(description='Sort by:', options=('^SPX', '^IXIC', '^DJI', '^VIX', 'SPY', '^RUT'…

In [72]:
# Sort the correlation matrix by a specific column Sum
sorted_correlation = transposed_measures_df.corr().abs()
sorted_correlation["Sum"]=sorted_correlation.sum()
sorted_correlation.sort_values(by='Sum', ascending=True)
sorted_correlation

Ticker,^SPX,^IXIC,^DJI,^VIX,SPY,^RUT,CL=F,GC=F,CME,Sum
Ticker,,,,,,,,,,
^SPX,1.000000,0.950104,0.366224,0.664821,0.999152,0.560287,0.858721,0.866292,0.221519,6.487120
^IXIC,0.950104,1.000000,0.553263,0.840868,0.956504,0.673231,0.913531,0.966524,0.030304,6.884329
^DJI,0.366224,0.553263,1.000000,0.896403,0.358737,0.958502,0.255968,0.726835,0.818355,5.934287
^VIX,0.664821,0.840868,0.896403,1.000000,0.668674,0.900821,0.655045,0.929188,0.563937,7.119757
SPY,0.999152,0.956504,0.358737,0.668674,1.000000,0.546587,0.878810,0.870867,0.223812,6.503143
^RUT,0.560287,0.673231,0.958502,0.900821,0.546587,1.000000,0.345366,0.801786,0.638255,6.424835
CL=F,0.858721,0.913531,0.255968,0.655045,0.878810,0.345366,1.000000,0.817190,0.186405,5.911037
GC=F,0.866292,0.966524,0.726835,0.929188,0.870867,0.801786,0.817190,1.000000,0.256379,7.235061
CME,0.221519,0.030304,0.818355,0.563937,0.223812,0.638255,0.186405,0.256379,1.000000,3.938966


In [73]:
# Covariance of the risk measures and sorting Sum
sorted_covariance = transposed_measures_df.cov().abs()
sorted_covariance["sum"]=sorted_covariance.sum()
sorted_covariance.sort_values(by='sum', ascending=True)

Ticker,^SPX,^IXIC,^DJI,^VIX,SPY,^RUT,CL=F,GC=F,CME,sum
Ticker,,,,,,,,,,
CL=F,0.071853,0.079560,0.024897,0.134819,0.072287,0.030393,0.083465,0.055228,0.021752,0.574255
GC=F,0.058693,0.068157,0.057244,0.154850,0.058002,0.057132,0.055228,0.054722,0.024225,0.588254
CME,0.025915,0.003690,0.111288,0.162276,0.025739,0.078530,0.021752,0.024225,0.163152,0.616566
SPY,0.082392,0.082095,0.034388,0.135629,0.081063,0.047404,0.072287,0.058002,0.025739,0.618999
^SPX,0.083885,0.082953,0.035711,0.137175,0.082392,0.049431,0.071853,0.058693,0.025915,0.628008
^IXIC,0.082953,0.090874,0.056152,0.180582,0.082095,0.061820,0.079560,0.068157,0.003690,0.705883
^RUT,0.049431,0.061820,0.098299,0.195483,0.047404,0.092787,0.030393,0.057132,0.078530,0.711278
^DJI,0.035711,0.056152,0.113351,0.215002,0.034388,0.098299,0.024897,0.057244,0.111288,0.746332
^VIX,0.137175,0.180582,0.215002,0.507521,0.135629,0.195483,0.134819,0.154850,0.162276,1.823336


In [ ]:
# The factor which is affecting the given stock volalitity is ????